In [34]:
# Dowload datasets

import os
from dotenv import load_dotenv
load_dotenv() # API Keys are stored in .env file

os.makedirs("datasets", exist_ok=True)

#======================================
# Load D-Fire dataset from Google Drive
#!pip install gdown
import gdown 
from zipfile import ZipFile
df_url = "https://drive.google.com/uc?id=19LSrZHYQqJSdKgH8Mtlgg7-i-L3eRhbh"  # smoke = 0, fire = 1
df_destination = "datasets\\D-Fire.zip"
df_directory = "datasets\\D-Fire"

if not os.path.isdir(df_directory):
    if not os.path.isfile(df_destination):
        print("Downloading dataset...", end='\r')
        gdown.download(df_url, df_destination, quiet=False)
        print("D-Fire dataset downloaded")
    else:
        with ZipFile(df_destination, 'r') as zip_ref:
            print("Unpacking..." , end="\r")
            zip_ref.extractall(df_directory)
            print("Files extracted")
print("D-Fire downloaded")

#======================================
# Load FASDD Dataset from Kaggle
#!pip install kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

k_dataset = 'cookiecacheqq/fasdd-cv' # smoke = 1, fire = 0 !
k_path = 'datasets'
k_dir = 'datasets\\FASDD_CV'
kapi = KaggleApi()

print("Downloading FASDD_CV")
if not os.path.isdir("datasets\\FASDD_CV\\FASDD_CV"):
    kapi.dataset_download_files(k_dataset, k_path)
    print("Extracting...")
    with ZipFile(k_path + "\\fasdd-cv.zip", 'r') as zip_ref:
        zip_ref.extractall(k_dir)
print("FASDD_CV extracted")

# Convert to YOLO Format and swap labels
import re

def swap_labels(data):
    d = re.sub("0 ", "2 ", data)
    d = re.sub("1 ", "0 ", d)
    d = re.sub("2 ", "1 ", d)
    return d

r = "datasets\\FASDD_CV_Converted"
#   1 - ensure filetree exists
os.makedirs(r, exist_ok=True)
for d in ["train", "test", "valid"]:
    os.makedirs(os.path.join(r, d), exist_ok=True)
    for dd in ["images",  "labels"]:
        os.makedirs(os.path.join(r, d, dd), exist_ok=True)

src = "datasets\\FASDD_CV\\FASDD_CV\\FASDD_CV\\annotations\\YOLO_CV"
img_src = "datasets\\FASDD_CV\\FASDD_CV\\FASDD_CV\\images"
for f, d in zip(["test.txt", "train.txt", "val.txt"], ["test", "train", "valid"]):
    with open(os.path.join(src, f), 'r') as llist:
        labels = llist.readlines()
    for l in labels:
        img_name = l.split("/")[-1].strip()
        l_name = re.sub(".jpg", ".txt", img_name)
        #  2 - copy label files and swap indexes
        with open(os.path.join(src, "labels", l_name), 'r') as lab:
            data = lab.read()
        with open(os.path.join(r, d, "labels", l_name), 'w') as lab:
            lab.write(swap_labels(data))
        # 3 - link img files
        os.symlink(os.path.join(img_src, d, "images", img_name), os.path.join(r, d, "images", img_name))

#======================================
# Load datasets from Roboflow
#!pip install roboflow
from roboflow import Roboflow

roboflow_api_key = os.getenv("RF_API_KEY")
rf = Roboflow(api_key=roboflow_api_key)

# Download roboflow datasets
os.chdir("datasets")
rf_fas = rf.workspace("data2").project("fire-smoke-detection-ua3dm").version(2).download("yolov11") # fire = 0, smoke = 1 !
rf_wfs = rf.workspace("brad-dwyer").project("wildfire-smoke").version(1).download("yolov11") # smoke = 0
print("Roboflow datasets downloaded")
os.chdir("..")

# Swap labels for rf_fas 

r = "datasets\\fire-smoke-detection-2_Converted"
#   1 - ensure filetree exists
os.makedirs(r, exist_ok=True)
for d in ["train", "test", "valid"]:
    os.makedirs(os.path.join(r, d), exist_ok=True)
    for dd in ["images",  "labels"]:
        os.makedirs(os.path.join(r, d, dd), exist_ok=True)

src = "datasets\\fire-smoke-detection-2"
for d in ["test", "train", "valid"]:
    for l_name in os.listdir(os.path.join(src, d, "labels"):
        img_name = re.sub(".txt", ".jpg", l_name)
        #  2 - copy label files and swap indexes
        with open(os.path.join(src, d, "labels", l_name), 'r') as lab:
            data = lab.read()
        with open(os.path.join(r, d, "labels", l_name), 'w') as lab:
            lab.write(swap_labels(data))
        # 3 - link img files
        os.symlink(os.path.join(src, d, "images", img_name), os.path.join(r, d, "images", img_name))



D-Fire downloaded
Dataset URL: https://www.kaggle.com/datasets/cookiecacheqq/fasdd-cv
FASDD_CV Downloaded
FASDD_CV extracted
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
Roboflow datasets downloaded


In [42]:

# Build data.yaml
yaml_data = f"""path: {os.getcwd() + "\\datasets"}
train: 
- D-Fire\\train
- FASDD_CV_Converted\\train
- fire-smoke-detection-2_Converted\\train
- Wildfire-Smoke-1\\train
val: 
- D-Fire\\test
- FASDD_CV_Converted\\valid
- fire-smoke-detection-2_Converted\\valid
test: 
- fire-smoke-detection\\test
- Wildfire-Smoke-1\\test

nc: 2
names: ['smoke', 'fire']
"""

with open("datasets\\data.yaml", 'w') as file:
    file.write(yaml_data)

In [ ]:
# Generate dataset metrics

def dataset_metrics(path): 
    # count images containing, count bboxes
    for tld in os.listdir(path):
        

In [38]:
# Start Tensorflow dashboard
%reload_ext tensorboard
%tensorboard --logdir runs/detect

In [ ]:
# Load pretrained YOLO model for transfer learning
#!pip install ultralytics
from ultralytics import YOLO
import os

# Training options
opts = {
    "data": "datasets\\data.yaml",
    "batch": 256,
    "val": True,
    "optimizer": "auto",
    "pretrained": True,
    "freeze": 11, # transfer learning
    "imgsz": 224, # larger is better for texture recognition but slow
    "plots": True,
    "workers": 1, # fix for old Nvidia GPU
}

# Train model to establish baseline performance
model = YOLO("yolo11n.pt")
model.train(**opts, epochs=50, name="yolo_v11n_224px")

New https://pypi.org/project/ultralytics/8.3.55 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.48  Python-3.12.7 torch-2.6.0.dev20241207+cu124 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=datasets\data.yaml, epochs=50, time=None, patience=100, batch=256, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=1, project=None, name=yolo_v11n_224px3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

train: Scanning E:\Documents\jupyter\FireAndSmokeDetection\datasets\D-Fire\train\labels.cache... 17737 images, 7833 bacIOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)


val: Scanning E:\Documents\jupyter\FireAndSmokeDetection\datasets\D-Fire\test\labels.cache... 4306 images, 2005 backgroIOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




Plotting labels to runs\detect\yolo_v11n_224px3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.002), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 224 train, 224 val
Using 1 dataloader workers
Logging results to runs\detect\yolo_v11n_224px3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.66G      2.046      3.656      1.677        653        224:  21%|██▏       | 15/70 [02:06<09:20, 10.

In [2]:
from ultralytics import YOLO

# Load best-fit model
model = YOLO("runs/detect/yolo_v11n_defaults/weights/best.pt")

In [ ]:
# MODEL SIZE REDUCTION

# Prune model
from torch.nn.utils import prune

ratio = 0.25

for name, module in model.named_modules():
    if ".conv" in name:
        print(f"Pruning layer {name}...")
        prune.l1_unstructured(module, name='weight', amount=ratio)
        prune.remove(module, 'weight')
print("Pruning done")

#TODO:
# Loop Fusion
# Kernel Tuning Optimizations
# FP16 Quantization

# Fine-tune after pruning0
model.train(**opts | {"freeze": False, "epochs":5, "name":"yolo_v11n_pruned"}) #override optimizer and lr
model.save(f'pruned_model.pt')

In [5]:
# Test on random inputs from dataset
import random
import os

random.seed()
num_imgs = 5

img_dir = "datasets/Combined" + "/test/images"
random_imgs = [random.choice(os.listdir(img_dir)) for x in range(num_imgs)]
results = model.predict([os.path.join(img_dir, im) for im in random_imgs])

for result in results:
    #boxes = result.boxes
    result.show()


0: 640x640 (no detections), 21.9ms
1: 640x640 (no detections), 21.9ms
2: 640x640 1 smoke, 21.9ms
3: 640x640 1 smoke, 21.9ms
4: 640x640 1 smoke, 21.9ms
5: 640x640 2 smokes, 6 fires, 21.9ms
6: 640x640 (no detections), 21.9ms
7: 640x640 2 smokes, 4 fires, 21.9ms
8: 640x640 4 fires, 21.9ms
9: 640x640 (no detections), 21.9ms
Speed: 5.5ms preprocess, 21.9ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)
